In [1]:
import spacy
import pandas as pd
from spacy.matcher import Matcher

In [2]:
# todo: set as env variable for raw main table
rootpath = "/hdd/work/d4ad_standardization/"
filepath = "./D4AD_Standardization/data/raw/etpl_all_programsJune3.xls"

columns = [
    "NAME",
    "NAME_1",
    "DESCRIPTION",
    "PREREQUISITES",
    "FEATURESDESCRIPTION",
    "STREET1",
    "CITY",
    "STATE",
    "ZIP",
    "WEBSITE",
    "COUNTY",
    "NONGOVAPPROVAL",
    "STATECOMMENTS",
    "CIPCODE"
]

df = pd.read_excel(rootpath + filepath, usecols=columns)
df.PREREQUISITES.fillna(' ', inplace=True) # space so that matches match

nlp = spacy.load("en_core_web_lg")
print('done')

done


In [3]:
# for test dev purposes, let's focus on a really small subset
BATCH_SIZE = 50

N = 20
random_state = 42
small_df = df.sample(n=N, random_state=random_state)
print('done')

done


In [20]:
def batches_of_docs(df, column_index=0, nlp=nlp, batch_size=BATCH_SIZE, disable=["parser","ner", "entity_linker"]):
    yield nlp.pipe(df.iloc[:,column_index].values,
                   batch_size=batch_size,
                   disable=disable)

def contents_of(matches, doc, matcher_spans_content=False):
    # if matcher_spans_content=False then the matcher
    # indicates where content is not and we span the
    # regions excluded by the matcher
    if not matcher_spans_content:
        match_start = 0
        for match in matches:
            match_end = match[1]
            if match[1] != -1:
                yield doc[match_start:match_end]
            match_start = match[2]
        if match_start != -1:
            yield doc[match_start:]


patterns =\
    [
        # these break up small_df.iloc[0] into unstandardized tokens
        #[{'POS': 'PUNCT'}],  # fails in later samples
        [{'POS': 'CCONJ'}],
        # modifiction that breaks up small_df.iloc[7]
        [{'ORTH': '/'}],
        # modifiction that combines small_df.iloc[15], [1]
        [{'ORTH': ','}],
        # modifiction seen generally past 50 or os
        [{'ORTH': ';'}],
        # TODO: fix this to work, i could be special casing too early/improperly
        # modifiction seen random_state*2
        [{'IS_SPACE': True}], # captures present spaces after tokenizations
    ]

matcher = Matcher(nlp.vocab)
matcher.add("DoNotStandardize", patterns)

interimpath = "./D4AD_Standardization/data/interim/"

content_is = "prereqs.csv"

import os 
the_df = df # could be small_df


# We write out batches of prereqs to disk for downstream analyses, class induction
# note: I am not sure why batch_number doesn't increment with nlp.pipe yeilds in batches of docs
for batch_number, a_batch in enumerate(batches_of_docs(the_df, batch_size= 50, column_index=10)):
    number_docs_per_batch = 50
    content_path = rootpath + interimpath + "{}_{}".format(batch_number, content_is)
    for doc_number, (doc, matches) in enumerate(matcher.pipe(a_batch, return_matches=True, batch_size=50)):
        direct_doc_index = batch_number*number_docs_per_batch + doc_number

        print('\t ... adding ', doc.text[:80], '...', 'doc_number ', doc_number, ' batch number ', batch_number)

        content_exists = not os.path.isfile(content_path)
        with open(content_path, 'a') as csv: # append to dataframe containing content from batch_number docs
            pd.DataFrame(
                data=\
                    {
                        'content': contents_of(matches, doc),
                        'CIPCODE': the_df.iloc[direct_doc_index].CIPCODE,
                        'PREREQUISITES': the_df.iloc[direct_doc_index].PREREQUISITES,                    
                        'batch_number': batch_number,
                        'doc_number': doc_number
                    }
            ).to_csv(
                csv,bbb
                index = False,
                chunksize = 10000,
                header=content_exists
            )

 have batch 
 in doc string
	 ... adding  High School Diploma/G.E.D. or Ability To Benefit ... doc_number  0  batch number  0
 in doc string
	 ... adding  High School Diploma/G.E.D. or Ability To Benefit, keyboarding - 25 WPM ... doc_number  1  batch number  0
 in doc string
	 ... adding  High School Diploma or G.E.D. or Ability To Benefit, work history ... doc_number  2  batch number  0
 in doc string
	 ... adding  High School Diploma or G.E.D. or Ability To Benefit, strong work history ... doc_number  3  batch number  0
 in doc string
	 ... adding  High School Diploma or G.E.D. or Ability To Benefit ... doc_number  4  batch number  0
 in doc string
	 ... adding  None ... doc_number  5  batch number  0
 in doc string
	 ... adding  None ... doc_number  6  batch number  0
 in doc string
	 ... adding  High School Diploma or GED ... doc_number  7  batch number  0
 in doc string
	 ... adding  None ... doc_number  8  batch number  0
 in doc string
	 ... adding  None ... doc_number  9  batch

 in doc string
	 ... adding  96 liberal arts credits and a GPA of 3.0+ ... doc_number  110  batch number  0
 in doc string
	 ... adding  GED or High School diploma ... doc_number  111  batch number  0
 in doc string
	 ... adding  HS diploma, GED ... doc_number  112  batch number  0
 in doc string
	 ... adding  Basic Computer Skill ... doc_number  113  batch number  0
 in doc string
	 ... adding  MS-Office/Word, Excel,Access. ... doc_number  114  batch number  0
 in doc string
	 ... adding  Reading comprehension 10th Grade level ... doc_number  115  batch number  0
 in doc string
	 ... adding  MS-Office/Word, Excel,Access. ... doc_number  116  batch number  0
 in doc string
	 ... adding  Basic Computer Skill. A+ / Network+/MCP ... doc_number  117  batch number  0
 in doc string
	 ... adding  Basic Computer Skill. A+ / Network+/MCP ... doc_number  118  batch number  0
 in doc string
	 ... adding  Basic Computer Skill. A+ ... doc_number  119  batch number  0
 in doc string
	 ... adding  H

 in doc string
	 ... adding  Word processing skills (25 words per minute) ... doc_number  227  batch number  0
 in doc string
	 ... adding  8th grade reading level, good manual dexterity and hand eye coordination ... doc_number  228  batch number  0
 in doc string
	 ... adding  Driver's License, 18 yrs old, pass DOT physical & drug test, 9th grade reading ... doc_number  229  batch number  0
 in doc string
	 ... adding  High School Diploma or GED ... doc_number  230  batch number  0
 in doc string
	 ... adding  Must have H.S.Diploma or GED, if not, must pass Wonderlic Test ... doc_number  231  batch number  0
 in doc string
	 ... adding  Juniors and seniors in participating school districts ... doc_number  232  batch number  0
 in doc string
	 ... adding  College Application and Accuplacer Testing ... doc_number  233  batch number  0
 in doc string
	 ... adding  SCC Application and Accuplacer Testing ... doc_number  234  batch number  0
 in doc string
	 ... adding  Must be at least 18 

 in doc string
	 ... adding  At discretion of company ... doc_number  343  batch number  0
 in doc string
	 ... adding  At discretion of company ... doc_number  344  batch number  0
 in doc string
	 ... adding  NJ Driving License, No more than 3 points on their NJ Driving License ... doc_number  345  batch number  0
 in doc string
	 ... adding  Mainframe or IBM Programmer, entry or jr. database administrator. ... doc_number  346  batch number  0
 in doc string
	 ... adding  Basic windows & keyboarding ... doc_number  347  batch number  0
 in doc string
	 ... adding  None ... doc_number  348  batch number  0
 in doc string
	 ... adding  None ... doc_number  349  batch number  0
 in doc string
	 ... adding  Discretion of employer ... doc_number  350  batch number  0
 in doc string
	 ... adding  Women or Minority intersted in construction career ... doc_number  351  batch number  0
 in doc string
	 ... adding  A+, Network Plus, Network + ... doc_number  352  batch number  0
 in doc string

	 ... adding  Knowledge of Windows and Office Products. Ability to run SQL and Pl/SQL queries ... doc_number  459  batch number  0
 in doc string
	 ... adding  Pass PC AGE Apt. Test : Have PC ... doc_number  460  batch number  0
 in doc string
	 ... adding  Knowledge of Windows and Office Products. Networking Essentials ... doc_number  461  batch number  0
 in doc string
	 ... adding  High School Diploma or GED ... doc_number  462  batch number  0
 in doc string
	 ... adding  Knowledge of Windows and Office Products ... doc_number  463  batch number  0
 in doc string
	 ... adding  Pass PC AGE Apt. Test. Have PC ... doc_number  464  batch number  0
 in doc string
	 ... adding  High school diploma or GED ... doc_number  465  batch number  0
 in doc string
	 ... adding  Essentially None. Prefer knowledge of Windows ... doc_number  466  batch number  0
 in doc string
	 ... adding  Knowledge of Windows and Office Products ... doc_number  467  batch number  0
 in doc string
	 ... adding  Pas

...

 in doc string
	 ... adding  Must have at least 2-3 years of professional work experience ... doc_number  24181  batch number  0
 in doc string
	 ... adding    ... doc_number  24182  batch number  0
 in doc string
	 ... adding  High School Diploma or GED ... doc_number  24183  batch number  0
 in doc string
	 ... adding  High School Diploma or GED ... doc_number  24184  batch number  0
 in doc string
	 ... adding  High School Diploma or GED ... doc_number  24185  batch number  0
 in doc string
	 ... adding  High School Diploma or GED ... doc_number  24186  batch number  0
 in doc string
	 ... adding  High School Diploma or GED ... doc_number  24187  batch number  0
 in doc string
	 ... adding  General working knowledge in the logistics, materials and supply chain managemen ... doc_number  24188  batch number  0
 in doc string
	 ... adding  General working knowledge in the logistics, materials and supply chain managemen ... doc_number  24189  batch number  0
 in doc string
	 ... adding 

 in doc string
	 ... adding  HS Diploma or GED ... doc_number  24298  batch number  0
 in doc string
	 ... adding  Must be able to perfrom safely in a shop environment. must be able to luft 50 lb ... doc_number  24299  batch number  0
 in doc string
	 ... adding  A student must have an Associate or Bachelor's Degree or higher in Information T ... doc_number  24300  batch number  0
 in doc string
	 ... adding    ... doc_number  24301  batch number  0
 in doc string
	 ... adding    ... doc_number  24302  batch number  0
 in doc string
	 ... adding  High School Diploma or GED and Official College Transcript ... doc_number  24303  batch number  0
 in doc string
	 ... adding  Four-year degree. 4,500 hours leading and directing projects. 35 hours of projec ... doc_number  24304  batch number  0
 in doc string
	 ... adding  High School Diploma or GED ... doc_number  24305  batch number  0
 in doc string
	 ... adding    ... doc_number  24306  batch number  0
 in doc string
	 ... adding  HS Dip

 in doc string
	 ... adding  High School Diploma or equivalent ... doc_number  24412  batch number  0
 in doc string
	 ... adding  0 ... doc_number  24413  batch number  0
 in doc string
	 ... adding  0 ... doc_number  24414  batch number  0
 in doc string
	 ... adding  9th grade  ... doc_number  24415  batch number  0
 in doc string
	 ... adding  Entrance Exam, HS Diploma or equivalent.  Students should purchase own books. ... doc_number  24416  batch number  0
 in doc string
	 ... adding  none ... doc_number  24417  batch number  0
 in doc string
	 ... adding  none ... doc_number  24418  batch number  0
 in doc string
	 ... adding  No Prerequisites ... doc_number  24419  batch number  0
 in doc string
	 ... adding  Individuals age 14-21 living with a physical , intellectual or mental health dis ... doc_number  24420  batch number  0
 in doc string
	 ... adding  Must be 21 years of age or older, driving abstract without multiple violations o ... doc_number  24421  batch number  0
 in 

	 ... adding    ... doc_number  24535  batch number  0
 in doc string
	 ... adding    ... doc_number  24536  batch number  0
 in doc string
	 ... adding    ... doc_number  24537  batch number  0
 in doc string
	 ... adding    ... doc_number  24538  batch number  0
 in doc string
	 ... adding    ... doc_number  24539  batch number  0
 in doc string
	 ... adding  8th grade level ... doc_number  24540  batch number  0
 in doc string
	 ... adding  High School Diploma or GED & Working knowledge of Windows Server ... doc_number  24541  batch number  0
 in doc string
	 ... adding  Knowledge of Supply Chain Management ... doc_number  24542  batch number  0
 in doc string
	 ... adding    ... doc_number  24543  batch number  0
 in doc string
	 ... adding  High School Diploma or GED ... doc_number  24544  batch number  0
 in doc string
	 ... adding  High School Diploma or GED ... doc_number  24545  batch number  0
 in doc string
	 ... adding  High School Diploma or GED ... doc_number  24546  batc

 in doc string
	 ... adding  High School Diploma or GED ... doc_number  24656  batch number  0
 in doc string
	 ... adding    ... doc_number  24657  batch number  0
 in doc string
	 ... adding    ... doc_number  24658  batch number  0
 in doc string
	 ... adding    ... doc_number  24659  batch number  0
 in doc string
	 ... adding  HS Diploma or HSE ... doc_number  24660  batch number  0
 in doc string
	 ... adding  None ... doc_number  24661  batch number  0
 in doc string
	 ... adding  High School Diploma/GED ... doc_number  24662  batch number  0
 in doc string
	 ... adding  High School Diploma or GED ... doc_number  24663  batch number  0
 in doc string
	 ... adding  High School Diploma or GED ... doc_number  24664  batch number  0
 in doc string
	 ... adding    ... doc_number  24665  batch number  0
 in doc string
	 ... adding  Student between 14-21 years old who has a disability, and is currently enrolled  ... doc_number  24666  batch number  0


In [ ]:
patterns =\
    [
        [{'LOWER': 'suspend'}],
        [{'LOWER': 'remove'}],
    ]

matcher = Matcher(nlp.vocab)
matcher.add("Suspended", patterns)

interimpath = "./D4AD_Standardization/data/interim/"

content_is = "state_comments.csv"

import os 
the_df = df.sample(n=N, random_state=42) # could be small_df
the_df.dropna(subset=['STATECOMMENTS'], inplace=True)  # only look at rows with content


# We write out batches of prereqs to disk for downstream analyses, class induction
# note: I am not sure why batch_number doesn't increment with nlp.pipe yields in batches of docs
for batch_number, a_batch in enumerate(batches_of_docs(the_df, batch_size= 50, column_index=8)):
    number_docs_per_batch = 50
    content_path = rootpath + interimpath + "{}_{}".format(batch_number, content_is)
    for doc_number, (doc, matches) in enumerate(matcher.pipe(a_batch, return_matches=True, batch_size=50)):
        direct_doc_index = batch_number*number_docs_per_batch + doc_number

        print('\t ... adding ', doc.text[:80], '...', 'doc_number ', doc_number, ' batch number ', batch_number)

        content_exists = not os.path.isfile(content_path)
        with open(content_path, 'a') as csv: # append to dataframe containing content from batch_number docs
            pd.DataFrame(
                data=\
                    {
                        'content': contents_of(matches, doc),
                        'CIPCODE': the_df.iloc[direct_doc_index].CIPCODE,
                        'STATECOMMENTS': the_df.iloc[direct_doc_index].STATECOMMENTS,                    
                        'batch_number': batch_number,
                        'doc_number': doc_number
                    }
            ).to_csv(
                csv,
                index = False,
                chunksize = 10000,
                header=content_exists
            )